In [13]:
import pandas as pd
import numpy as np
import random
import os
import random
import shutil

train 1210
test 302

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
import shutil
import time
import pandas as pd
import random

random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

C:\Users\msba7028gpu\anaconda3\envs\d2l\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\msba7028gpu\anaconda3\envs\d2l\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [34]:
random.seed(100)
new_data_dir = r"C:/Users/msba7028gpu/Downloads/train_valid_test"   

# image augmentation

In [24]:
random.seed(100)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0),  
                                 ratio=(3.0/4.0, 4.0/3.0)),
    transforms.RandomHorizontalFlip(),
    # randomly change brightness, contrast, saturation and hue
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.5),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])


transform_test = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# data loading

In [25]:
train_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'train'),
                                            transform=transform_train)
test_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'test'),
                                            transform=transform_test)
batch_size = 32   #batch size can change
train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)  # shuffle=False

# k-fold

In [35]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.CrossEntropyLoss()

dataset = train_ds

num_epochs=30
batch_size=32
#k=5
#splits=KFold(n_splits=k,shuffle=True,random_state=42)
#foldperf={}

# Define Model

In [27]:
def get_net():
    net = torchvision.models.vgg16(pretrained=True)
    net.classifier[-1] = nn.Linear(in_features=4096, out_features=8)
    nn.init.xavier_uniform_(net.classifier[-1].weight)
    return net

# CV

In [28]:
def train_batch_ch13(net, X, y, loss, trainer, devices):
    """Train for a minibatch with mutiple GPUs (defined in Chapter 13)."""
    if isinstance(X, list):
        # Required for BERT fine-tuning (to be covered later)
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    net.train()
    trainer.zero_grad()
    pred = net(X)
    l = loss(pred, y)
    l.sum().backward()
    trainer.step()
    train_loss_sum = l.sum()
    train_acc_sum = d2l.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

In [29]:
from d2l import torch as d2l
def train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus()):
    """Train a model with mutiple GPUs (defined in Chapter 13)."""
    timer, num_batches = d2l.Timer(), len(train_iter)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples,
        # no. of predictions
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = train_batch_ch13(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
    
    loss = metric[0] / metric[2]
    train_acc = metric[1] / metric[3]

    return loss,train_acc,test_acc

In [36]:
def train_fine_tuning(net, learning_rate, trainloader, testloader, batch_size=32, num_epochs=30,
                      param_group=True):
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["classifier.6.weight", "classifier.6.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.classifier[-1].parameters(),
                                    'lr': learning_rate * 10}],
                                lr=learning_rate, weight_decay=0.01)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.01)
    
    loss,train_acc,test_acc = train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus())
    return loss,train_acc,test_acc

In [37]:
def cv_result(learning_rate,batch_size,num_epochs,param_group=True):
    loss_ls =[]
    train_acc_ls =[]
    test_acc_ls = []
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

        print('Fold {}'.format(fold + 1))

        #all test there means validation

        train_sampler = SubsetRandomSampler(train_idx) 
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
        
        net = get_net()

        loss,train_acc,test_acc = train_fine_tuning(net, learning_rate, train_loader, test_loader, batch_size=32, num_epochs=30,
                          param_group=True)

        loss_ls.append(loss)
        train_acc_ls.append(train_acc)
        test_acc_ls.append(test_acc)
        print('loss: ',loss)
        print('train: ',train_acc)
        print('test: ',test_acc)
    return loss_ls,train_acc_ls,test_acc_ls

In [14]:
#the first combination of hyperparameters
loss_ls,train_acc_ls,test_acc_ls = cv_result(0.0001,32,25,param_group=True)
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

Fold 1
loss:  0.5136792986846167
train:  0.8256198347107438
test:  0.7906976744186046
Fold 2
loss:  0.5303957671173348
train:  0.8231404958677686
test:  0.8039867109634552
Fold 3
loss:  0.5440735903653231
train:  0.8066115702479338
test:  0.7940199335548173
Fold 4
loss:  0.5095627729557762
train:  0.8214876033057851
test:  0.7906976744186046
Fold 5
loss:  0.5344766656229318
train:  0.8289256198347107
test:  0.7807308970099668
5fold_average_loss: 0.526, 5fold_average_train_accuracy: 0.821, 5fold_average_test_accuracy: 0.792


In [17]:
#the second combination of hyperparameters
loss_ls,train_acc_ls,test_acc_ls = cv_result(0.0001,64,25,param_group=True)
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

Fold 1
loss:  0.529166266543806
train:  0.8115702479338843
test:  0.7873754152823921
Fold 2
loss:  0.5246661320205562
train:  0.8231404958677686
test:  0.7973421926910299
Fold 3
loss:  0.5080617794320603
train:  0.8297520661157025
test:  0.7973421926910299
Fold 4
loss:  0.517758906183164
train:  0.8330578512396695
test:  0.7940199335548173
Fold 5
loss:  0.4994344439388307
train:  0.8314049586776859
test:  0.7641196013289037
5fold_average_loss: 0.516, 5fold_average_train_accuracy: 0.826, 5fold_average_test_accuracy: 0.788


In [38]:
#the third combination of hyperparameters
loss_ls,train_acc_ls,test_acc_ls = cv_result(0.0001,32,30,param_group=True)
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

Fold 1
loss:  0.46655266580502847
train:  0.8512396694214877
test:  0.7807308970099668
Fold 2
loss:  0.4650608433179619
train:  0.8471074380165289
test:  0.8205980066445183
Fold 3
loss:  0.47752330441120244
train:  0.8347107438016529
test:  0.7674418604651163
Fold 4
loss:  0.45781230532433376
train:  0.8413223140495868
test:  0.7840531561461794
Fold 5
loss:  0.5005391215489916
train:  0.8330578512396695
test:  0.813953488372093
5fold_average_loss: 0.473, 5fold_average_train_accuracy: 0.841, 5fold_average_test_accuracy: 0.793
